# Read data from Redis

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import redis
import json
import datetime as dt

Connect to Redis

In [2]:
store = redis.Redis()

## Read data

In [11]:
personId = 2
rawList = store.lrange(f'personData{personId}', 0, -1) # -1 is the last element

type(rawList)
# rawList

data = [json.loads(d.decode()) for d in rawList] # Decode each JSON
data

[{'birthdate': '1976',
  'disabled': True,
  'firstname': 'Elżbieta',
  'id': 12,
  'lastname': 'Kochalska',
  'trace': {'id': 13562005062019,
   'name': 'ela',
   'sensors': [{'anomaly': False, 'id': 0, 'value': 1023},
    {'anomaly': False, 'id': 1, 'value': 799},
    {'anomaly': False, 'id': 2, 'value': 1023},
    {'anomaly': False, 'id': 3, 'value': 12},
    {'anomaly': False, 'id': 4, 'value': 12},
    {'anomaly': False, 'id': 5, 'value': 12}]},
  'timestamp': 1577145841.550726},
 {'birthdate': '1976',
  'disabled': True,
  'firstname': 'Elżbieta',
  'id': 12,
  'lastname': 'Kochalska',
  'trace': {'id': 13561805062019,
   'name': 'ela',
   'sensors': [{'anomaly': False, 'id': 0, 'value': 17},
    {'anomaly': False, 'id': 1, 'value': 246},
    {'anomaly': False, 'id': 2, 'value': 12},
    {'anomaly': False, 'id': 3, 'value': 1023},
    {'anomaly': False, 'id': 4, 'value': 1023},
    {'anomaly': False, 'id': 5, 'value': 1023}]},
  'timestamp': 1577145840.309111},
 {'birthdate': '19

## Attemt to put data to df

In [33]:
current_id = 1

key = f'personData{current_id}'

rawList = store.lrange(key, 0, 5)
data = [json.loads(d.decode()) for d in rawList]

values = {'time': [] , 'sensor_0': [], 'sensor_1': [], 'sensor_2': [], 'sensor_3': [], 'sensor_4': [], 'sensor_5': []}

for value in data:
    values['time'].append(dt.datetime.fromtimestamp(value['timestamp']))
    sensors = value['trace']['sensors']
    
    for s in sensors:
        id = s['id']
        key = f'sensor_{id}'
        values[key].append(s['value'])
#     print(sensors)
    
    
pd.DataFrame(values)

,time,sensor_0,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2019-12-24 01:24:12.326738,1023,749,1023,1023,271,1023
1,2019-12-24 01:24:11.048903,1023,657,1023,1023,247,1023
2,2019-12-24 01:24:09.776034,1023,732,1023,1023,253,1023
3,2019-12-24 01:24:08.529523,1023,691,1023,1023,253,1023
4,2019-12-24 01:24:07.246190,1023,633,1023,1023,248,1023
5,2019-12-24 01:24:05.942032,1023,721,1023,1023,249,1023


Not yet successfull

In [29]:
data[0]['trace']['sensors']

[{'anomaly': False, 'id': 0, 'value': 370},
 {'anomaly': False, 'id': 1, 'value': 363},
 {'anomaly': False, 'id': 2, 'value': 14},
 {'anomaly': False, 'id': 3, 'value': 1023},
 {'anomaly': False, 'id': 4, 'value': 1023},
 {'anomaly': False, 'id': 5, 'value': 1023}]

In [36]:
tmp = [[(d['trace']['sensors'][i]['value'], d['trace']['sensors'][i]['anomaly']) for i in range(6)] for d in data]

tmp

[[(370, False),
  (363, False),
  (14, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(335, False),
  (456, False),
  (15, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(380, False),
  (497, False),
  (15, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(133, False),
  (341, False),
  (14, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(510, False),
  (418, False),
  (13, False),
  (1023, False),
  (1023, False),
  (910, False)],
 [(368, False),
  (256, False),
  (13, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(113, False),
  (364, False),
  (14, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(1023, False),
  (1023, False),
  (1023, False),
  (13, False),
  (12, False),
  (13, False)],
 [(1023, False),
  (1023, False),
  (16, False),
  (983, False),
  (393, False),
  (1023, False)],
 [(1023, False),
  (1023, False),
  (1023, False),
  (12, False),
  (13, False),
  (13, False)],
 [(1023, False)

In [45]:
fields = {f'Value {i}': [] for i in range(1, 6)}
fields.update({f'Anomaly {i}': [] for i in range(1, 6)})
fields

{'Value 1': [],
 'Value 2': [],
 'Value 3': [],
 'Value 4': [],
 'Value 5': [],
 'Anomaly 1': [],
 'Anomaly 2': [],
 'Anomaly 3': [],
 'Anomaly 4': [],
 'Anomaly 5': []}

In [69]:
flattened = [[]] * 12 
print(flattened)

for row in tmp:
    i = 0
    for sensor in row:
        flattened[i].append(sensor[0])
#         print(i)
#         print(sensor[0])
        i += 1

        flattened[i].append(sensor[1])
#         print(i)
#         print(sensor[1])
        i += 1
#         print(i)
        
flattened[1]

[[], [], [], [], [], [], [], [], [], [], [], []]


[370,
 False,
 363,
 False,
 14,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 335,
 False,
 456,
 False,
 15,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 380,
 False,
 497,
 False,
 15,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 133,
 False,
 341,
 False,
 14,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 510,
 False,
 418,
 False,
 13,
 False,
 1023,
 False,
 1023,
 False,
 910,
 False,
 368,
 False,
 256,
 False,
 13,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 113,
 False,
 364,
 False,
 14,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 13,
 False,
 12,
 False,
 13,
 False,
 1023,
 False,
 1023,
 False,
 16,
 False,
 983,
 False,
 393,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 12,
 False,
 13,
 False,
 13,
 False,
 1023,
 False,
 725,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 350,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 207,
 False

In [21]:
formated = {"1": [i] for i in range(5)}
formated

{'1': [4]}

In [18]:
tmp = {"1": [4, 2], "2": [7, 8]}

pd.DataFrame(tmp)

,1,2
0,4,7
1,2,8
